# Model Prediction

In [1]:
import csv
from datetime import datetime
from lstm import Machine
import pandas as pd
import tensorflow as tf
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import cufflinks as cf

MODEL_PATH = "data/lstm_model_openPrice.hd5"
CSV_PATH = "data/binance-BTCUSD.csv"
TRAIN_TEST_RATE = 0.85
FIELD = 'openPrice'

cf.go_offline()
init_notebook_mode(connected=True)

with open(CSV_PATH) as fin:
    reader = [dict(item) for item in csv.DictReader(fin)]
machine = Machine(reader, FIELD)
(_, _, X_test, Y_test) = machine.split_data(TRAIN_TEST_RATE)

machine.field = 'openTime'
times = machine.filtered_data
times = times[int(len(machine.split_seq(30)) * TRAIN_TEST_RATE) + 29:]
times = [datetime.utcfromtimestamp(t / 1000.0) for t in times.values]
machine.field = FIELD

model = tf.keras.models.load_model(MODEL_PATH)
pred = machine.scaler.inverse_transform(model.predict(X_test, batch_size=2))
pred = pred.reshape(pred.shape[0])

Y_test = machine.scaler.inverse_transform(Y_test).reshape(Y_test.shape[0])

plots = pd.DataFrame({
    'Predictions': pred,
    'Real Prices': Y_test,
}, index=times)
iplot(plots.iplot(
    asFigure=True,
    kind='scatter',
    xTitle='Date',
    yTitle='BTC Price',
    title='BTC Price Predictions'
))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
